In [1]:
import pandas as pd
import requests
import time
import urllib.parse


# Function to get Spotify access token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_data = auth_response.json()
    return auth_data['access_token']

# Function to search for a track and get its ID
def search_track(track_name, artist_name, token):
    query = f"track:{track_name} artist:{artist_name}"
    query_encoded = urllib.parse.quote(query)
    url = f"https://api.spotify.com/v1/search?q={query_encoded}&type=track&limit=1"
    response = requests.get(url, headers={
        'Authorization': f'Bearer {token}'
    })
    json_data = response.json()
    try:
        first_result = json_data['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except (KeyError, IndexError):
        return None

# Function to get track details
def get_track_details(track_id, token):
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    response = requests.get(url, headers={
        'Authorization': f'Bearer {token}'
    })
    json_data = response.json()
    try:
        image_url = json_data['album']['images'][0]['url']
        return image_url
    except (KeyError, IndexError):
        return None

# Your Spotify API Credentials
client_id = '0d7b5d511b8846709dfd76d46c0b302e'
client_secret = '679e5b71a69e41cfa4d19b5cedc3d6a6'

# Get Access Token
access_token = get_spotify_token(client_id, client_secret)

# Read your DataFrame
try:
    df_spotify = pd.read_csv('/content/spotify-2023.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    df_spotify = pd.read_csv('/content/spotify-2023.csv', encoding='utf-16')

# Loop through each row to get track details and add to DataFrame
cover_urls = []
for i, row in df_spotify.iterrows():
    track_id = search_track(row['track_name'], row['artist(s)_name'], access_token)
    if track_id:
        image_url = get_track_details(track_id, access_token)
        cover_urls.append(image_url)
    else:
        # Retry with modifications if no cover is found
        artists = row['artist(s)_name'].split(',')
        image_url = None
        for artist in artists:
            track_id = search_track(row['track_name'], artist.strip(), access_token)
            if track_id:
                image_url = get_track_details(track_id, access_token)
                if image_url:
                    break
        cover_urls.append(image_url)

# Add the cover URLs as a new column to the DataFrame
df_spotify['cover_url'] = cover_urls

# Save the updated DataFrame
df_spotify.to_csv('spotify-2023-with-cover-urls.csv', index=False)

# Display the first few rows of the updated DataFrame
print(df_spotify.head())


                            track_name    artist(s)_name  artist_count  \
0  Seven (feat. Latto) (Explicit Ver.)  Latto, Jung Kook             2   
1                                 LALA       Myke Towers             1   
2                              vampire    Olivia Rodrigo             1   
3                         Cruel Summer      Taylor Swift             1   
4                       WHERE SHE GOES         Bad Bunny             1   

   released_year  released_month  released_day  in_spotify_playlists  \
0           2023               7            14                   553   
1           2023               3            23                  1474   
2           2023               6            30                  1397   
3           2019               8            23                  7858   
4           2023               5            18                  3133   

   in_spotify_charts    streams  in_apple_playlists  ...  key   mode  \
0                147  141381703                  4